In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf

In [1]:
from InitialPositions import *
from WaveFunctions import *
from Hamiltonians import *
from Optimizers import *
from sampler import Sampler
from metropolis import metro_util
from particles import Particles

In [2]:
# problem specific multi-use parameters
HO_shape = [1, 1, 1]
hard_core_r = 0.00433

config = {
    "name"        : "demo",
    "particles"   : 100,
    "dimensions"  : 3,
    
    "steps"       : {"sampled" : int(1.0E5), "equil" : int(1.0E3)},
    "metropolis"  : {"name" : "importance", "step_length" : 0.01},
    "optimizer"   : {"name" : "grid", "start" : 0.4, "end" : 0.6, "step" : 0.05},
    
    "initial"     : {"name" : "randomUniform", "radius" : hard_core_r},
    "wavefunc"    : {"name" : "simpleGaussian", "params" : 0.5, "HO shape" : HO_shape},
    "hamiltonian" : {"name" : "HO", "omega" : 1, "HO shape" : HO_shape},
    
    #"waveFunc"   : {"name" : "boson-corr", "alpha" : 0.5, "HO shape" : HO_shape, "hard-core-r" : hard_core_r}
}

In [3]:
initPos     = initPos_util.getInit(config)
metro_step  = metro_util(config)
hamiltonian = ham_util.getHam(config)


sampler   = Sampler(config)
waveFunc  = wf_util.getWf(config)
particles = initPos()

In [ ]:
pos2 = initPos()
pos = pos2.getPositions()

In [ ]:
%%time
for i in range(1000):
    metro_step(pos2, waveFunc)

In [ ]:
%%time
for i in range(100):
    waveFunc.paramDer(pos)

In [ ]:

pos = particles.getPositions()

def test(b, pos):
    b = 0
    for i in range(10000):
        b += waveFunc.paramDer(pos)
    return b

In [ ]:
%%time
test(b, pos)

In [ ]:
%%time
#tf function call?
for i in range(100):
    metro_step(particles, waveFunc)
#tf function call with gradienttape and two wavefunctions?
for i in range(100):
    metro_step(particles, waveFunc)
    sampler.sample(particles, waveFunc, hamiltonian)

In [ ]:
%%time
s = np.random.uniform(size=(100, 100))
test(s)

In [ ]:
def test(s):
    for i in range(1000):
        s += np.random.uniform(size=(100, 100))**2
    return s

In [ ]:
import numpy as np
positions = np.random.uniform(size=(100, 100))

In [ ]:
%%time
b = positions * 0
for i in range(1000):
    b += positions * positions

In [ ]:
hamiltonian.potential(particles)

In [ ]:
import tensorflow as tf
tf.random.uniform(shape = (3,), minval = -1, maxval = 1, dtype = tf.float64)

In [ ]:
%%time
for i in range(config["steps"]["equil"]):
    metro_step(particles, waveFunc)
#tf function call with gradienttape and two wavefunctions?
for i in range(config["steps"]["sampled"]):
    metro_step(particles, waveFunc)
    sampler.sample(particles, waveFunc, hamiltonian)